#### Luis Ramirez
#### Problem Set 1
In this assingment I work on problems from a bioinformatics text by: Dr. Greg Caporaso

#### Chapter 2.1: Pairwise Sequence Alingment####
We explore sequence similarity

In [2]:
%pylab inline 
import numpy as np
from IPython.core.display import HTML
from IPython.core import page
page.page = print


Populating the interactive namespace from numpy and matplotlib


In [10]:
from scipy.spatial.distance import hamming
from skbio import DNA


r1 = DNA("ACCCAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGACACACGGGGATTAA")
r2 = DNA("ACCGAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGAGACACGGCGATTAA")
q1 = DNA("TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA")

print(r1)
print(r2)
print(q1)

ACCCAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGACACACGGGGATTAA
ACCGAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGAGACACGGCGATTAA
TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA


Storinng three DNA sequences as variables. Then we compute the hamming distance between them using the hamming() function.
    

In [11]:
%psource hamming

print(hamming(r1,q1))
print(hamming(r2,q1))

def hamming(u, v, w=None):
    """
    Compute the Hamming distance between two 1-D arrays.

    The Hamming distance between 1-D arrays `u` and `v`, is simply the
    proportion of disagreeing components in `u` and `v`. If `u` and `v` are
    boolean vectors, the Hamming distance is

    .. math::

       \\frac{c_{01} + c_{10}}{n}

    where :math:`c_{ij}` is the number of occurrences of
    :math:`\\mathtt{u[k]} = i` and :math:`\\mathtt{v[k]} = j` for
    :math:`k < n`.

    Parameters
    ----------
    u : (N,) array_like
        Input array.
    v : (N,) array_like
        Input array.
    w : (N,) array_like, optional
        The weights for each value in `u` and `v`. Default is None,
        which gives each value a weight of 1.0

    Returns
    -------
    hamming : double
        The Hamming distance between vectors `u` and `v`.

    Examples
    --------
    >>> from scipy.spatial import distance
    >>> distance.hamming([1, 0, 0], [0, 1, 0])
    0.66666666666666663
    >>>

we can observe that q1 has a smaller distance to r1 than it does to r2, so q1 is more similar to r1 than r2. However, it is not always this simple.

We assumed that only substitution events have occured, meaning one DNA base was substituted with another. So we define q2, which is similar to q1 except that a single base has been deleted at the beginning of the sequence. And a single base has been inserted at the end of the sequence. 

In [12]:
q2 = DNA("TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCAT")

print(q1)
print(q2)

print(hamming(r1, q2))

TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA
TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCAT
0.7288135593220338


We note how the distance increased with two small changes. The deletion even at the beginning of `q2` shifted that sequence relative to `r1`, this may have resulted in the many bases "downstream" of the deleted bases being different. The large distance may not be biologically justified. 

What would be nice to do is to have a way to indicate that a deletion has occured in `q2`. Let's define `q3` where we use a `-` character to indicate a deletion with respect to `r1`. This might be a more reasonable result, lets see...

In [13]:
q3 = DNA("-TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA")

print(q1)
print(q3)

print(hamming(r1, q3))

TTCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA
-TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA
0.2542372881355932


What we did created a pairwise alignment of `r1` and `q3`. In other words, we have aligned positions to maximize the similarity between the two sequences. We will refer `-` as gap characters. 

We can observe the alignment ourselves if we print the output. 

In [14]:
print(r1)
print(q3)

ACCCAGGTTAACGGTGACCAGGTACCAGAAGGGTACCAGGTAGGACACACGGGGATTAA
-TCCAGGTAAACGGTGACCAGGTACCAGTTGCGTTTGTTGTAGGAGACACGGGGACCCA


#### 2.1.1: Answering what is a sequence alignemnt? ####
Over the course of biological evolution, a DNA sequence changes. Frequently due to random errors in replication. These errors are known as mutations. Here are a few examples of mutations: Substitutions, where one base (or amino acid) is replaced. Insertions, where one or more contiguous bases are inserted into a sequence. Deletions, where one or more contiguous bases are deleted from a sequence. Ancestral DNA sequences, over time, might evolve into two derived sequences. When two ore more sequences are derived from a single ancestral sequence. Such sequences are said to be **homologs** of one another.   

So, the goal of pairwise seqence alignment is to generate a hypothesis about which sequence positions are derived from a common ancestral sequence position. This hypotheis is developed by aligning the sequences to one another by inserting gaps as necessary such that the similiarity is maximixed. This approach is known as maximum parsimony (Occam's Razor), where the most likely explanation is the simplist one. 

Since we do not know what the ancestral sequence was, when we encounter a gap in a pairwise sequence alignment we are unsure if a deletion or insertion occured in either of the sequences. These types of events will be refered to as **indel**.

#### 2.1.2 Practicing a procedure for aligning a pair of sequences ####
 We practice a basic informatics algorithms for aligning a pair of sequences. First we define the two sequences, `seq1` and `seq2`.

In [15]:
seq1 = DNA("ACCGGTGGAACCGGTAACACCCAC")
seq2 = DNA("ACCGGTAACCGGTTAACACCCAC")

We are going to use a function called `show_F` to display a table that we are using to develop our alignment. We can also view the source code

In [16]:
from BioinformaticsCode import show_F
from tabulate import tabulate
%psource show_F

from skbio.alignment import _tabular_msa
from tabulate import tabulate
def show_F(h_sequence, v_sequence, data, hide_zeros=False, nonzero_val=None):

    rows = []
    col_headers = [c.decode('UTF-8') for c in h_sequence.values]
    row_headers = [c.decode('UTF-8') for c in v_sequence.values]
    pad_headers: Union[bool, Any] = data.shape == (len(row_headers) + 1, len(col_headers) + 1)
    if pad_headers:
        row_headers = [" "] + row_headers
        col_headers = [" "] + col_headers
    for h, d in zip(row_headers, data):
        current_row = ["<b>%s</b>" % h]
        for e in d:
            if e == 0:
                if hide_zeros:
                    current_row.append('')
                else:
                    current_row.append(0)
            else:
                if nonzero_val is not None:
                    current_row.append(nonzero_val)
                else:
                    current_row.append(e)
        rows.append(current_row)
    return tabulate(rows, headers=c

First we create a blank matrix where the rows and columns represent the positions in the sequences. Initialize with zeros as follows:

In [17]:
num_rows = len(seq2)
num_cols = len(seq1)
data = np.zeros(shape=(num_rows, num_cols), dtype=np.int)

HTML(show_F.show_F(seq1, seq2, data))

,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# creating a copy of our data matrix to work with, so we leave the original untouched
summed_data = data.copy()
# iterate over the cells in our data matrix, starting in the second row and the second column
for i in range(1, summed_data.shape[0]):
    for j in range(1, summed_data.shape[1]):
        if summed_data[i,j] > 0:
            summed_data[i, j] += summed_data[i - 1][j - 1]
print("The longest diagonal is %d characters long" % summed_data.max())
HTML(show_F.show_F(seq1,seq2, summed_data, hide_zeros= True))

The longest diagonal is 10 characters long


,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,2,1,,,,,,,,2,1,,,,,,2,,2,1,1,,2
C,,1,3,,,,,,,,1,3,,,,,,1,,1,3,2,,1
G,,,,4,1,,1,1,,,,,4,1,,,,,,,,,,
G,,,,1,5,,1,2,,,,,1,5,,,,,,,,,,
T,,,,,,6,,,,,,,,,6,,,,,,,,,
A,1,,,,,,,,1,1,,,,,,7,1,,1,,,,1,
A,1,,,,,,,,1,2,,,,,,1,8,,1,,,,1,
C,,2,1,,,,,,,,3,1,,,,,,9,,2,1,1,,2
C,,1,3,,,,,,,,1,4,,,,,,1,,1,3,2,,1


In [20]:
for row_number,row_character in enumerate(seq2):
    for col_number, col_character in enumerate(seq1):
        if row_character == col_character:
            data[row_number,col_number] = 1
HTML(show_F.show_F(seq1,seq2,data,hide_zeros=True))

,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1
G,,,,1,1,,1,1,,,,,1,1,,,,,,,,,,
G,,,,1,1,,1,1,,,,,1,1,,,,,,,,,,
T,,,,,,1,,,,,,,,,1,,,,,,,,,
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1


Next we'll identify the longest stretches of non-zero characters, which will be refered to as diagonals

In [21]:
# creating a copy of our data matrix to work with, so we leave the original untouched
summed_data = data.copy()
# iterate over the cells in our data matrix, starting in the second row and the second column
for i in range(1, summed_data.shape[0]):
    for j in range(1, summed_data.shape[1]):
        if summed_data[i,j] > 0:
            summed_data[i, j] += summed_data[i - 1][j - 1]
print("The longest diagonal is %d characters long" % summed_data.max())
HTML(show_F.show_F(seq1,seq2, summed_data, hide_zeros= True))

The longest diagonal is 10 characters long


,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,2,1,,,,,,,,2,1,,,,,,2,,2,1,1,,2
C,,1,3,,,,,,,,1,3,,,,,,1,,1,3,2,,1
G,,,,4,1,,1,1,,,,,4,1,,,,,,,,,,
G,,,,1,5,,1,2,,,,,1,5,,,,,,,,,,
T,,,,,,6,,,,,,,,,6,,,,,,,,,
A,1,,,,,,,,1,1,,,,,,7,1,,1,,,,1,
A,1,,,,,,,,1,2,,,,,,1,8,,1,,,,1,
C,,2,1,,,,,,,,3,1,,,,,,9,,2,1,1,,2
C,,1,3,,,,,,,,1,4,,,,,,1,,1,3,2,,1


Now we transcribe a possible alignment that arises from this process. We can use the following algorithm. 
- Start with the longest diagonal and trace it backwards to transcribe the alignment by writing down the characters from each of the two sequences at every row and column corresponding to the diagonal that you are following. 
- When we encounter a break in the diagonal, we find the next longest diagonal that starts in a cell that is up and/or to the left of the cell when the previous diagonal we were following ends.
- For every cell that we move straight upwards, we would insert a gap in the sequence on the horizontal axis of the matrix. For every cell that you move straight leftwards, you'd insert a gap in the sequence on the horizontal axis of your matrix
- For every vell that you move straight leftwards, you'd insert a gap in the sequence on the vertical axis of your matrix.

Finally here is one possible alignment assuming we follow the algorithm correctly:
- **ACCGGTGGAACCGG-TAACACCCAC**
- **ACCGGT--AACCGGTTAACACCCAC**